## 사용 패키지

In [89]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [90]:
from pycaret.regression import *

## 데이터 로드

In [102]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/sample_submission.csv')

## 데이터

In [103]:
train.head()

일자 요일  본사정원수  본사휴가자수  본사출장자수  본사시간외근무명령서승인건수  현본사소속재택근무자수  \
0  2016-02-01  월   2601      50     150             238          0.0   
1  2016-02-02  화   2601      50     173             319          0.0   
2  2016-02-03  수   2601      56     180             111          0.0   
3  2016-02-04  목   2601     104     220             355          0.0   
4  2016-02-05  금   2601     278     181              34          0.0   

                                                조식메뉴  \
0  모닝롤/찐빵  우유/두유/주스 계란후라이  호두죽/쌀밥 (쌀:국내산) 된장찌개  쥐...   
1  모닝롤/단호박샌드  우유/두유/주스 계란후라이  팥죽/쌀밥 (쌀:국내산) 호박젓국찌...   
2  모닝롤/베이글  우유/두유/주스 계란후라이  표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...   
3  모닝롤/토마토샌드  우유/두유/주스 계란후라이  닭죽/쌀밥 (쌀,닭:국내산) 근대국...   
4  모닝롤/와플  우유/두유/주스 계란후라이  쇠고기죽/쌀밥 (쌀:국내산) 재첩국  방...   

                                                중식메뉴  \
0  쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개  쇠불고기 (쇠고기:호주산) 계란찜 ...   
1  쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개  가자미튀김  모둠소세지구이  마늘쫑무...   
2  카레덮밥 (쌀,현미흑미:국내산) 팽이장국  치킨핑거 (닭고기:국내산) 쫄면야채무침 ...   
3  쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국  주꾸미볶음  부추전  시금치나물  ...   
4  쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국  돈육씨앗강정 (돼지고기:국내산) 우엉잡채...   

                                                석식메뉴     중식계    석식계  
0  쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장  자반고등어구이  두부조림  건파래무침 ...  1039.0  331.0  
1  콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국  유산슬 (쇠고기:호주산) 아삭고추무...   867.0  560.0  
2  쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개  황태양념구이 (황태:러시아산) 고기...  1017.0  573.0  
3  미니김밥*겨자장 (쌀,현미흑미:국내산) 우동  멕시칸샐러드  군고구마  무피클  포...   978.0  525.0  
4  쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...   925.0  330.0

1. MinMaxScaler
2. 휴일 전, 휴일 후 column 추가
3. 기온, 강수량 column 추가



In [104]:
from sklearn.preprocessing import MinMaxScaler

In [105]:
train['본사정원수'] = train['본사정원수']-train['본사휴가자수']-train['본사출장자수']-train['현본사소속재택근무자수']

In [112]:
test['본사정원수'] = test['본사정원수']-test['본사휴가자수']-test['본사출장자수']-test['현본사소속재택근무자수']

In [106]:
scaler = MinMaxScaler()

a = ['본사정원수','본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수']

In [123]:
train['휴일전'] = train.apply(lambda x: 1 if x['요일'] == 4 else 0 , axis = 1) 
train['휴일후'] = train.apply(lambda x: 1 if x['요일'] == 0 else 0 , axis = 1)

In [108]:
train = train.drop(columns = ['본사휴가자수', '본사출장자수','현본사소속재택근무자수'])

In [110]:
train[['본사정원수','본사시간외근무명령서승인건수']] = scaler.fit_transform(train[['본사정원수', '본사시간외근무명령서승인건수']])

In [124]:
train.head()

일자  요일     본사정원수  본사시간외근무명령서승인건수  \
0  2016-02-01   0  0.664300        0.227969   
1  2016-02-02   1  0.649451        0.305556   
2  2016-02-03   2  0.641059        0.106322   
3  2016-02-04   3  0.584248        0.340038   
4  2016-02-05   4  0.497095        0.032567   

                                                조식메뉴  \
0  모닝롤/찐빵  우유/두유/주스 계란후라이  호두죽/쌀밥 (쌀:국내산) 된장찌개  쥐...   
1  모닝롤/단호박샌드  우유/두유/주스 계란후라이  팥죽/쌀밥 (쌀:국내산) 호박젓국찌...   
2  모닝롤/베이글  우유/두유/주스 계란후라이  표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...   
3  모닝롤/토마토샌드  우유/두유/주스 계란후라이  닭죽/쌀밥 (쌀,닭:국내산) 근대국...   
4  모닝롤/와플  우유/두유/주스 계란후라이  쇠고기죽/쌀밥 (쌀:국내산) 재첩국  방...   

                                                중식메뉴  \
0  쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개  쇠불고기 (쇠고기:호주산) 계란찜 ...   
1  쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개  가자미튀김  모둠소세지구이  마늘쫑무...   
2  카레덮밥 (쌀,현미흑미:국내산) 팽이장국  치킨핑거 (닭고기:국내산) 쫄면야채무침 ...   
3  쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국  주꾸미볶음  부추전  시금치나물  ...   
4  쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국  돈육씨앗강정 (돼지고기:국내산) 우엉잡채...   

                                                석식메뉴     중식계    석식계  휴일전  휴일후  
0  쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장  자반고등어구이  두부조림  건파래무침 ...  1039.0  331.0    0    1  
1  콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국  유산슬 (쇠고기:호주산) 아삭고추무...   867.0  560.0    0    0  
2  쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개  황태양념구이 (황태:러시아산) 고기...  1017.0  573.0    0    0  
3  미니김밥*겨자장 (쌀,현미흑미:국내산) 우동  멕시칸샐러드  군고구마  무피클  포...   978.0  525.0    0    0  
4  쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...   925.0  330.0    1    0

In [40]:
test.head()

일자  요일  본사정원수  본사휴가자수  본사출장자수  본사시간외근무명령서승인건수  현본사소속재택근무자수  \
0  2021-01-27   2   2983      88     182               5        358.0   
1  2021-01-28   3   2983     104     212             409        348.0   
2  2021-01-29   4   2983     270     249               0        294.0   
3  2021-02-01   0   2924     108     154             538        322.0   
4  2021-02-02   1   2924      62     186             455        314.0   

                                                조식메뉴  \
0  모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...   
1  모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...   
2  모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...   
3  모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...   
4  모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...   

                                                중식메뉴  \
0  쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...   
1  쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...   
2  쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...   
3  쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...   
4  쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...   

                                            석식메뉴  
0  흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치   
1            충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지   
2            흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치   
3           흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치   
4       흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치

In [113]:
test = test.drop(columns = ['본사휴가자수', '본사출장자수','현본사소속재택근무자수'])

In [121]:
test[['본사정원수', '본사시간외근무명령서승인건수']] = scaler.transform(test[['본사정원수', '본사시간외근무명령서승인건수']])

test['휴일전'] = test.apply(lambda x: 1 if x['요일'] == 4 else 0 , axis = 1) 
test['휴일후'] = test.apply(lambda x: 1 if x['요일'] == 0 else 0 , axis = 1)

In [122]:
test.head(10)

일자  요일     본사정원수  본사시간외근무명령서승인건수  \
0  2021-01-27   2 -0.886304    4.394086e-09   
1  2021-01-28   3 -0.886304    3.594362e-07   
2  2021-01-29   4 -0.886304    0.000000e+00   
3  2021-02-01   0 -0.886304    4.728036e-07   
4  2021-02-02   1 -0.886304    3.998618e-07   
5  2021-02-03   2 -0.886304    4.394086e-09   
6  2021-02-04   3 -0.886304    4.183169e-07   
7  2021-02-05   4 -0.886304    0.000000e+00   
8  2021-02-08   0 -0.886304    6.063838e-07   
9  2021-02-09   1 -0.886304    4.763189e-07   

                                                조식메뉴  \
0  모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...   
1  모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...   
2  모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...   
3  모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...   
4  모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...   
5  모닝롤/게살모닝샌드 우유/주스 계란후라이/찐계란 소고기죽/흑미밥 시래기된장국 베이컨...   
6  모닝롤/사과파이 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 아욱국 새송이버섯곤약장...   
7  모닝롤/앙버터모닝빵 우유/주스 계란후라이/찐계란 고구마죽/흑미밥 옹심이국 머위나물무...   
8  모닝롤/스콘 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 꽃게탕 근대나물무침 연두부...   
9  모닝롤/치즈팡샌드 우유/주스 계란후라이/찐계란 팥죽/흑미밥 맑은버섯국 시금치나물무침...   

                                                중식메뉴  \
0  쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...   
1  쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...   
2  쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...   
3  쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...   
4  쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...   
5   쌀밥/흑미밥/찰현미밥 아욱국 매콤해물볶음 감자조림 미나리나물 포기김치 콥샐러드*렌치D    
6  쌀밥/차조밥/찰현미밥 설렁탕 고등어김치말이찜 볼어묵굴소스볶음 브로콜리숙회*초장 석박...   
7  쌀밥/흑미밥/찰현미밥 북엇국 닭볶음탕 채소전*장 솎음열무나물무침 포기김치 양상추샐러...   
8  쌀밥/흑미밥/찰현미밥 감자양파국 돈수육*씨앗쌈장 매콤어묵볶음 콩나물파채무침 포기김치...   
9  쌀밥/기장밥/찰현미밥 장각백숙 적어양념장구이 채소스틱*쌈장 도라지오이초무침 겉절이김...   

                                            석식메뉴  휴일전  휴일후  
0  흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치     0    0  
1            충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지     0    0  
2            흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치     1    0  
3           흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치     0    1  
4       흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치     0    0  
5   오므라이스 가쓰오장국 빌소세지구이*구운채소 단감치커리무침 양념고추지 겉절이김치     0    0  
6     흑미밥 계란파국 돈육두루치기 감자채파프리카볶음 세발나물오리엔탈무침 포기김치     0    0  
7       유부초밥/추가밥 온메밀소바 국물떡볶이 순대찜*소금 청경채겉절이 포기김치     1    0  
8                 흑미밥 냉이국 반반치킨 꼬막채소무침 청경채찜 포기김치     0    1  
9            흑미밥 미역국 매운소불고기 단호박두부탕수 메추리알장조림 석박지     0    0

In [10]:
submission.head()

일자  중식계  석식계
0  2021-01-27    0    0
1  2021-01-28    0    0
2  2021-01-29    0    0
3  2021-02-01    0    0
4  2021-02-02    0    0

## 요일을 숫자로 매핑

In [119]:
train['요일'] = train['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})
test['요일'] = test['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})

## 수치 데이터만 피쳐로 선택

In [125]:
train_data1 = train[['요일', '본사정원수', '본사시간외근무명령서승인건수','휴일전','휴일후','중식계']]
train_data2 = train[['요일', '본사정원수',  '본사시간외근무명령서승인건수','휴일전','휴일후','석식계']]

In [126]:
x_test = test[['요일', '본사정원수', '본사시간외근무명령서승인건수','휴일전','휴일후']]

## 점심과 저녁을 식수 인원을 예측할 2개 모델 생성

In [127]:
exp_reg1 = setup(train_data1, target = '중식계')

In [75]:
exp_reg2 = setup(train_data2, target = '석식계')

In [128]:
best = compare_models(sort = 'MAE')

In [77]:
ada = create_model('ada', cross_validation = False)
lgbm = create_model('lightgbm', cross_validation = False)
gbr = create_model('gbr', cross_validation = False)

MAE        MSE     RMSE      R2   RMSLE    MAPE
0  63.5339  8849.0247  94.0693  0.4805  1.0335  0.1266

In [78]:
tuned_gbr = tune_model(gbr, optimize = 'RMSE', n_iter = 10)
tuned_ada = tune_model(ada, optimize = 'RMSE', n_iter = 10)
tuned_lgbm = tune_model(lgbm, optimize = 'RMSE', n_iter = 10)

In [79]:
blender_specific = blend_models(estimator_list = [tuned_lgbm,tuned_ada,tuned_gbr], optimize = 'MAE')

In [80]:
final_model = finalize_model(blender_specific)

In [74]:
pred1 = predict_model(final_model, data = x_test)

In [81]:
pred2 = predict_model(final_model, data = x_test)

## 학습

## 추론

In [82]:
pred1

요일     본사정원수    본사출장자수  본사시간외근무명령서승인건수  현본사소속재택근무자수  휴일전  휴일후        Label
0    2  0.542614  0.418398        0.004789     0.671670    0    0   995.316894
1    3  0.542614  0.507418        0.391762     0.652908    0    0   957.093732
2    4  0.542614  0.617211        0.000000     0.551595    1    0   648.470355
3    0  0.458807  0.335312        0.515326     0.604128    0    1  1179.316505
4    1  0.458807  0.430267        0.435824     0.589118    0    0   972.945574
5    2  0.458807  0.468843        0.004789     0.536585    0    0   950.447270
6    3  0.458807  0.504451        0.455939     0.540338    0    0   925.793105
7    4  0.458807  0.626113        0.000000     0.480300    1    0   622.469417
8    0  0.458807  0.394659        0.660920     0.617261    0    1  1207.060242
9    1  0.458807  0.421365        0.519157     0.617261    0    0   981.228298
10   2  0.458807  0.275964        0.000000     0.437148    0    0   949.732337
11   0  0.458807  0.267062        0.761494     0.666041    0    1  1258.503209
12   1  0.458807  0.397626        0.780651     0.774859    0    0  1086.128604
13   2  0.458807  0.415430        0.002874     0.711069    0    0   990.577461
14   3  0.458807  0.611276        0.568966     0.634146    0    0   860.515528
15   4  0.458807  0.673591        0.000958     0.525328    1    0   607.059438
16   0  0.458807  0.462908        0.779693     0.463415    0    1  1208.414455
17   1  0.458807  0.471810        0.750000     0.437148    0    0   977.140044
18   2  0.458807  0.575668        0.002874     0.439024    0    0   822.812731
19   3  0.458807  0.626113        0.560345     0.440901    0    0   817.897407
20   4  0.458807  0.706231        0.000958     0.335835    1    0   561.343961
21   1  0.531250  0.370920        0.748084     0.465291    0    0  1028.622847
22   2  0.531250  0.456973        0.000958     0.547842    0    0   983.316975
23   3  0.531250  0.578635        0.714559     0.493433    0    0   915.589901
24   4  0.531250  0.640950        0.001916     0.424015    1    0   595.146783
25   0  0.531250  0.382789        0.899425     0.495310    0    1  1245.089636
26   1  0.531250  0.382789        0.960728     0.529081    0    0  1084.582954
27   2  0.531250  0.427300        0.000958     0.508443    0    0   932.953179
28   3  0.531250  0.540059        0.583333     0.489681    0    0   901.716477
29   4  0.531250  0.590504        0.000000     0.470919    1    0   640.572975
30   0  0.531250  0.317507        0.827586     0.589118    0    1  1263.418229
31   1  0.531250  0.400593        0.630268     0.752345    0    0  1049.286868
32   2  0.531250  0.448071        0.003831     0.696060    0    0   996.251109
33   3  0.531250  0.563798        0.498084     0.658537    0    0   924.182867
34   4  0.531250  0.635015        0.000000     0.499062    1    0   616.825828
35   0  0.531250  0.370920        0.677203     0.656660    0    1  1241.208005
36   1  0.531250  0.486647        0.659004     0.654784    0    0  1023.825511
37   2  0.531250  0.572700        0.000000     0.589118    0    0   916.852229
38   3  0.531250  0.649852        0.422414     0.557223    0    0   829.732792
39   4  0.531250  0.667656        0.000000     0.429644    1    0   606.154732
40   0  0.531250  0.406528        0.632184     0.596623    0    1  1230.076042
41   1  0.531250  0.465875        0.598659     0.587242    0    0  1005.339294
42   2  0.531250  0.522255        0.000000     0.594747    0    0   962.267709
43   3  0.528409  0.637982        0.377395     0.568480    0    0   836.013176
44   4  0.528409  0.685460        0.000000     0.420263    1    0   603.978834
45   0  0.528409  0.394659        0.674330     0.621013    0    1  1232.440257
46   1  0.528409  0.382789        0.609195     0.682927    0    0  1044.158052
47   2  0.528409  0.513353        0.000958     0.626642    0    0   976.360073
48   3  0.528409  0.584570        0.487548     0.607880    0    0   894.913269
49   4  0.528409  0.673591        0.000000     0.42964

## 결과 제출

In [83]:
submission['중식계'] = pred1['Label']
submission['석식계'] = pred2['Label']

In [84]:
submission.head()

일자          중식계         석식계
0  2021-01-27   995.316894  347.031022
1  2021-01-28   957.093732  439.834265
2  2021-01-29   648.470355  318.331590
3  2021-02-01  1179.316505  505.132939
4  2021-02-02   972.945574  453.697597

In [85]:
submission.to_csv('baseline.csv', index=False)